In [ ]:
!pip install tld

     |████████████████████████████████| 412 kB 5.0 MB/s 


# Loading the Quotes data in Google BigQuery (BQ)
The goal of this file is to load the quotebank quotes datasets (one dataset available for each year) from Google drive (list of compressed JSON files) into BQ.

The initial files to be loaded in BQ are in Google Drive. This python script opens by chunk the large yearly quotes files (so that they fit in memory) and then loads uploads them in BQ after some slight pre-processing.

It is heavy in the first place to upload everything to BQ but the advantages that the team can enjoy later on are major. In fact, being able to operate infinitely auto-scaling queries on the whole dataset at once is crucial for insighful preliminary analysis. Since the cloud architecture is auto-scaling it is easy to run such analysis. This is why we chose the intuitive and auto-scaling platform Google BigQuery as data warehouse and no-SQL platform.

In [ ]:
import bz2
import json
import pandas as pd

In [ ]:
from tld import get_tld

def get_domain(url):
    res = get_tld(url, as_object=True)
    return res.tld

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import json, os

# Google Cloud services
gcp_service_account_credentials_json_filename = '/content/epfl-course-f41b0ed796f9.json' #need to upload the json credential files to the root directory of the google colab files
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = gcp_service_account_credentials_json_filename
credentials = service_account.Credentials.from_service_account_file(gcp_service_account_credentials_json_filename, scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/drive'])
project_id = 'epfl-course'
bigquery_client = bigquery.Client(credentials=credentials, project=project_id)
bigquery_client = bigquery.Client()

#Execute a query in BQ
def bq_execute_query(query, mode="INTERACTIVE", wait=False, to_dataframe=False):
    job_config = bigquery.QueryJobConfig(priority="bigquery.QueryPriority.{}".format(mode)) # Run at BATCH priority, which won't count toward concurrent rate limit, otherwise INTERACTIVE.
    query_job = bigquery_client.query(query, job_config)
    if wait==True:
        print("Executed BQ query: ", query_job.result())
    if to_dataframe==True:
        return(query_job.to_dataframe())
    else:
        return(query_job)

#Upload a DF to BQ
def upload_df_to_bq_initial_upload(df, bq_destination_table, write_disposition="WRITE_APPEND"):
    #bq_table_name = "epfl-course.dataset.table"
    job_config = bigquery.LoadJobConfig(create_disposition="CREATE_IF_NEEDED", 
                                        write_disposition=write_disposition,
                                        schema=[
                                            bigquery.SchemaField("quids", "STRING"),
                                            bigquery.SchemaField("probas", "STRING"),
                                            bigquery.SchemaField("urls", "STRING"),
                                            bigquery.SchemaField("domains", "STRING"),
                                        ])
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    upload_df_to_bq_job = bigquery_client.load_table_from_dataframe(
        df, bq_destination_table, job_config = job_config)
    print("Uploaded DF to BQ: ",upload_df_to_bq_job.result()) 

#Upload a JSON to BQ
def upload_json_to_bq(json_object, bq_table):
    try:
        job_config = bigquery.LoadJobConfig()
        job_config.autodetect = False #Change to True if the table on BQ does not exits
        job_config.max_bad_records = 0
        job_config.ignore_unknown_values = True
        job_config.source_format = 'NEWLINE_DELIMITED_JSON'
        job_config.create_disposition= "CREATE_IF_NEEDED"
        job_config.write_disposition= "WRITE_APPEND"
        job = bigquery_client.load_table_from_file(json_object, bq_table, job_config = job_config)
        print("Loaded JSON to BQ table {} as job {}".format(bq_table, job.result()))
        assert job.job_type == 'load'
        assert job.state == 'DONE'
    except:
        print("ERROR Could not load JSON to BQ table {} as job {}".format(bq_table, job.result()))


In [ ]:
list_of_years_to_process = range(2015, 2020) #list of the years which data has to be send to Google BigQuery(BQ)

# Function uploads the JSON file to BQ and removes it from the disk to save storage
def upload_interim_file_to_bq(filename, year):
    print("uploading file: ", filename)
    df_to_upload = pd.read_json(filename, 
                                lines=True) #open file to upload to convert it from a JSON file to panda dataframe (df)
    bq_table = "epfl-course.ada_project.interim_data_{}".format(str(year))
    upload_df_to_bq_initial_upload(df_to_upload, 
                    bq_table) #upload the df to BQ !!table name as varible
    print("Uploaded df of shape: ", df_to_upload.shape)
    os.remove(filename) #remove the JSON file not to keep them on disk storage (since we are working on Google drive, disk storage is limited)

# Upload the data of one-year file to BQ
def upload_one_year_of_data(year):
    path_to_file = '/content/drive/MyDrive/EPFL HS21/Applied data analysis CS-401/Project/ADA team winner/Project Milestone 2/Quotebank/quotes-{}.json.bz2'.format(year)

    filename_out = 'quotes-{}-domains'.format(year)
    counter = 0
    number_of_rows_to_upload_per_batch = 100000 #reasonable threshold for file size upload to BQ (approx. 50mb per file uploaded)
    path_to_out_complete_previous = "" #a logical utility to compare the filename to the previous for-loop run
    with bz2.open(path_to_file, 'rb') as s_file:
        for instance in s_file: #iterates though every row of the 1-year file
            path_to_out_complete = filename_out + str(round(counter/number_of_rows_to_upload_per_batch)).zfill(10) + ".json" #filename of the file to be uploaded
            with open(path_to_out_complete, 'ab') as d_file: #open in append mode 'a' with binary encoding 'b'
                instance = json.loads(instance) # loading a sample
                urls = instance['urls'] # extracting list of links
                domains = []
                for url in urls:
                    tld = get_domain(url)
                    domains.append(tld)
                instance['domains'] = domains # updating the sample with domain name
                d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file

                #logic that allows to upload data to BQ by batches of "number_of_rows_to_upload_per_batch" rows
                counter += 1
                if path_to_out_complete_previous == path_to_out_complete: #case where file to upload is not yet complete
                  pass
                elif path_to_out_complete_previous == "": #first iteration case
                  path_to_out_complete_previous = path_to_out_complete
                else: # case when the file is complete with enough data to be uploaded
                  upload_interim_file_to_bq(path_to_out_complete_previous, year)
                  path_to_out_complete_previous = path_to_out_complete
    upload_interim_file_to_bq(path_to_out_complete, year) #upload the last file for the year

# Upload data from all years
for year in list_of_years_to_process:
  print("Processing data for year: ", year)
  upload_one_year_of_data(year)

Processing data for year:  2015
uploading file:  quotes-2015-domains0000000000.json


/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'quoteID'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Uploaded DF to BQ:  <google.cloud.bigquery.job.LoadJob object at 0x7f72d28731d0>
Uploaded df of shape:  (50001, 10)
uploading file:  quotes-2015-domains0000000001.json


/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'quoteID'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Uploaded DF to BQ:  <google.cloud.bigquery.job.LoadJob object at 0x7f72d27ee910>
Uploaded df of shape:  (99999, 10)
uploading file:  quotes-2015-domains0000000002.json


...

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'quoteID'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Uploaded DF to BQ:  <google.cloud.bigquery.job.LoadJob object at 0x7f72d2929b50>
Uploaded df of shape:  (100001, 10)
uploading file:  quotes-2015-domains0000000007.json


/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'quoteID'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Uploaded DF to BQ:  <google.cloud.bigquery.job.LoadJob object at 0x7f72d2814f10>
Uploaded df of shape:  (99999, 10)


KeyboardInterrupt: ignored

In [ ]:
# The previous block has been Keyboard interrupted in order to just show a couple logs. 
# The complete processing of all the files is too long to be shown in a single 
# code file notebook (here just to explain how it would work).